# dev-env v3.0 — **Tek Tıkla Geliştirme Ortamı**
**Code Server + Nexus + Forgejo** – **Sıfır Manuel, Her Yerde Çalışır**

![dev-env](https://img.shields.io/badge/dev--env-v3.0-brightgreen) ![Colab](https://img.shields.io/badge/Colab-Ready-orange) ![Docker](https://img.shields.io/badge/Docker-Ready-blue)

---

## Bu Notebook Ne Yapar?

1. **Tüm dosyaları otomatik oluşturur** (`docker-compose.yml`, `setup.sh`, vs.)
2. **Rastgele şifreler** üretir
3. **SSH anahtarı** oluşturur
4. **VS Code extensions** ekler
5. **`dev-env.zip`** paketini hazırlar
6. **Google Colab’da indirir!**

> **Hiçbir şey bilmenize gerek yok. Sadece hücreleri sırayla çalıştırın.**

---

## Nasıl Kullanılır?

| Ortam | Yapılacak |
|-------|---------|
| **Google Colab** | [Open in Colab](https://colab.research.google.com/github/kullanici/dev-env/blob/main/dev-env.ipynb) → Tüm hücreleri çalıştır |
| **Jupyter / VS Code** | `dev-env.ipynb` dosyasını aç → `Run All` |

---

## Sonuç

- `dev-env.zip` → İndir
- `unzip dev-env.zip -d dev-env`
- `cd dev-env && ./setup.sh`

**TAMAM!** 30 saniyede geliştirme ortamın hazır.

---

## Servisler

| Servis | URL | Kimlik |
|--------|-----|--------|
| **Code Server** | `http://localhost:8080` | Şifre: `.env` içinde |
| **Nexus** | `http://localhost:8081` | `admin` / `nexus-data/admin.password` |
| **Forgejo** | `http://localhost:3000` | İlk giriş: `admin` / `admin` |

---

## Sonraki Adım: HÜCRELERİ ÇALIŞTIR →


## 1. Gerekli Kütüphaneler & Yardımcı Fonksiyonlar

In [ ]:
import os, zipfile, stat, textwrap, json, secrets, string
from pathlib import Path
from IPython.display import display, Markdown, JSON
import shutil

# Yardımcı Fonksiyonlar
def rand_pass(length=16):
    chars = string.ascii_letters + string.digits + "!@#$%^&*"
    return ''.join(secrets.choice(chars) for _ in range(length))

def write_file(root, rel_path, content):
    p = root / rel_path
    p.parent.mkdir(parents=True, exist_ok=True)
    p.write_text(content, encoding='utf-8')

def make_executable(path):
    path.chmod(path.stat().st_mode | stat.S_IEXEC)

print("Hazır!")

## 2. Ortam ve Şifreler

In [ ]:
BASE = Path("/content/dev-env")  # Colab’da /content
BASE.mkdir(exist_ok=True)
shutil.rmtree(BASE, ignore_errors=True)  # Temizle (tekrar çalıştırılabilir)
BASE.mkdir(exist_ok=True)

CODE_PASS = rand_pass(16)

display(Markdown(f"**Code Server Şifresi:** `{CODE_PASS}`\n\n"))
display(Markdown(f"**Çalışma Klasörü:** `{BASE}`"))

## 3. Dosya İçerikleri (docker-compose, Dockerfile, setup.sh...)

In [ ]:
files = {
    # docker-compose.yml
    "docker-compose.yml": textwrap.dedent(f"""\
        version: "3.9"
        services:
          nexus:
            image: sonatype/nexus3:latest
            container_name: nexus
            restart: unless-stopped
            ports:
              - "${{NEXUS_PORT}}:8081"
            volumes:
              - ./nexus-data:/nexus-data
            healthcheck:
              test: ["CMD", "curl", "-f", "http://localhost:8081"]
              interval: 30s
              timeout: 10s
              retries: 5
              start_period: 60s
            networks: [devnet]

          code-server:
            build:
              context: .
              dockerfile: Dockerfile
            container_name: code-server
            environment:
              - PASSWORD=${{CODE_SERVER_PASSWORD}}
            ports:
              - "${{CODE_SERVER_PORT}}:8080"
            volumes:
              - ./projects:/home/coder/project
              - ./config:/home/coder/.config
              - ./rust:/home/coder/.cargo
              - ./rustup:/home/coder/.rustup
              - ./python:/home/coder/.local
              - ./pip-cache:/home/coder/.cache/pip
              - ./bash/.bashrc:/home/coder/.bashrc
              - ./bash/aliases.sh:/home/coder/.aliases
              - ./config/.ssh:/home/coder/.ssh:ro
            depends_on:
              nexus:
                condition: service_healthy
            healthcheck:
              test: ["CMD", "curl", "-f", "http://localhost:8080"]
              interval: 10s
              timeout: 5s
              retries: 3
            restart: unless-stopped
            networks: [devnet]

          forgejo:
            image: codeberg.org/forgejo/forgejo:8
            container_name: forgejo
            restart: unless-stopped
            environment:
              - USER_UID=1000
              - USER_GID=1000
              - FORGEJO__database__DB_TYPE=sqlite3
              - FORGEJO__server__HTTP_PORT=3000
              - FORGEJO__server__ROOT_URL=http://localhost:${{FORGEJO_PORT}}/
            ports:
              - "${{FORGEJO_PORT}}:3000"
            volumes:
              - ./forgejo-data:/data
            depends_on:
              - code-server
            networks: [devnet]

        networks:
          devnet:
            driver: bridge
    """),

    "Dockerfile": textwrap.dedent("""\
        FROM codercom/code-server:latest AS base

        USER root
        RUN apt-get update && \
            apt-get install -y curl git build-essential pkg-config libssl-dev libclang-dev python3-pip && \
            rm -rf /var/lib/apt/lists/*

        RUN curl https://sh.rustup.rs -sSf | sh -s -- -y && \
            . "$HOME/.cargo/env" && \
            rustup default stable && \
            rustup component add rustfmt clippy rust-analyzer

        RUN pip3 install --no-cache-dir --upgrade pip

        COPY extensions.txt /tmp/
        RUN while read ext; do code-server --install-extension "$ext" --force; done < /tmp/extensions.txt

        RUN chown -R coder:coder /home/coder
        ENV PATH="/home/coder/.cargo/bin:/home/coder/.local/bin:$PATH"
        ENV CARGO_NET_GIT_FETCH_WITH_CLI=true

        USER coder
        WORKDIR /home/coder/project
    """),

    "setup.sh": textwrap.dedent(f"""\
        #!/usr/bin/env bash
        set -euo pipefail

        echo "dev-env v3.0 — Code Server + Nexus + Forgejo"
        echo "=========================================="

        mkdir -p nexus-data projects config/rust config/rustup config/python config/pip-cache \
                 config/.ssh bash forgejo-data rust rustup python pip-cache

        if [ ! -f .env ]; then
          cat > .env <<EOF
        CODE_SERVER_PASSWORD={CODE_PASS}
        CODE_SERVER_PORT=8080
        NEXUS_PORT=8081
        FORGEJO_PORT=3000
        EOF
          echo ".env oluşturuldu (şifre: {CODE_PASS})"
        fi

        if [ ! -f config/.ssh/id_ed25519 ]; then
          ssh-keygen -t ed25519 -C "dev@local" -f config/.ssh/id_ed25519 -N "" -q
          echo "SSH anahtarı oluşturuldu"
          echo "Public key:"
          cat config/.ssh/id_ed25519.pub
        fi

        [ ! -f bash/.bashrc ] && cat > bash/.bashrc <<'EOF'
        PS1="\[\\e[01;34m\\]\\u@\\h:\\w\[\\e[00m\\]\$ "
        [ -f ~/.aliases ] && . ~/.aliases
        export PATH="$HOME/.cargo/bin:$HOME/.local/bin:$PATH"
        export CARGO_NET_GIT_FETCH_WITH_CLI=true
        [ -f "$HOME/.cargo/env" ] && . "$HOME/.cargo/env"
        EOF

        [ ! -f bash/aliases.sh ] && cat > bash/aliases.sh <<'EOF'
        alias ll='ls -alF'
        alias gs='git status'
        alias py='python3'
        alias r='cargo run'
        alias b='cargo build'
        alias t='cargo test'
        EOF

        chmod +x bash/aliases.sh

        cat > extensions.txt <<'EOF'
        ms-python.python
        rust-lang.rust-analyzer
        vadimcn.vscode-lldb
        serayuzgur.crates
        EOF

        echo "Docker Compose başlatılıyor..."
        docker compose up -d --build

        echo "\nKURULUM TAMAM!"
        echo "  Code Server : http://localhost:8080  (şifre: {CODE_PASS})"
        echo "  Nexus       : http://localhost:8081"
        echo "  Forgejo     : http://localhost:3000  (admin / admin)"
    """),

    ".env.example": textwrap.dedent("""\
        CODE_SERVER_PASSWORD=your_secure_password
        CODE_SERVER_PORT=8080
        NEXUS_PORT=8081
        FORGEJO_PORT=3000
    """),

    "extensions.txt": textwrap.dedent("""\
        ms-python.python
        rust-lang.rust-analyzer
        vadimcn.vscode-lldb
        serayuzgur.crates
    """),

    "VERSION.txt": "v3.0.0 - 2025-10-29\n+ Forgejo (Gitea yerine)\n+ Colab uyumlu\n+ Her adım ayrı hücre",

    "README.md": textwrap.dedent("""\
        # dev-env v3.0 — Tam Otomatik Geliştirme Ortamı

        **Code Server + Nexus + Forgejo** – Tek ZIP, sıfır manuel.

        ## Hızlı Başlangıç
        ```bash
        unzip dev-env.zip -d dev-env
        cd dev-env
        chmod +x setup.sh
        ./setup.sh
        ```

        ## Erişim
        - Code Server: http://localhost:8080
        - Nexus: http://localhost:8081
        - Forgejo: http://localhost:3000 (admin/admin)
    """)
}

print(f"{len(files)} dosya tanımlandı.")

## 4. Dosyaları Yaz & setup.sh’ı Çalıştırılabilir Yap

In [ ]:
for rel_path, content in files.items():
    write_file(BASE, rel_path, content)

setup_path = BASE / "setup.sh"
make_executable(setup_path)

display(Markdown("**Tüm dosyalar yazıldı!**"))

## 5. ZIP Oluştur

In [ ]:
ZIP_PATH = Path("/content/dev-env.zip")
with zipfile.ZipFile(ZIP_PATH, "w", compression=zipfile.ZIP_DEFLATED) as z:
    for root, _, fnames in os.walk(BASE):
        for f in fnames:
            fp = Path(root) / f
            arc = fp.relative_to(BASE.parent)
            z.write(fp, arc)

size_kb = ZIP_PATH.stat().st_size / 1024
display(Markdown(f"**ZIP hazır!** → `{ZIP_PATH}`\nBoyut: **{size_kb:.1f} KB**"))

## 6. İndir (Colab’da Otomatik)

In [ ]:
try:
    from google.colab import files
    files.download(str(ZIP_PATH))
    display(Markdown("**İndirme başladı!**"))
except ImportError:
    display(Markdown(f"**Colab dışı ortam.** ZIP burada: `{ZIP_PATH}`"))

# Sonuç
JSON({"zip_path": str(ZIP_PATH), "password": CODE_PASS, "version": "v3.0.0"})